## 計測システムのプロトタイプ

In [1]:
import adafruit_bno055
import busio# I2C用のインターフェースを使用するためのモジュール
import board

import threading
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
path = os.getcwd()
from IPython.display import display, clear_output
from collections import deque
from time import perf_counter
import ipywidgets as widgets
import time
import threading
import datetime


In [2]:
class measuremet:
    
    def __init__(self, BNO_UPDATE_FREQUENCY_HZ=10, INTERVAL=1000):
        
        self.COLUMNS = ["Time","euler_x", "euler_y", "euler_z", "gyro_x", "gyro_y", "gyro_z", "gravity_x", "gravity_y", "gravity_z",
                        "linear_accel_x", "linear_accel_y", "linear_accel_z","accel_x", "accel_y", "accel_z",
                        "quaternion_1", "quaternion_2", "quaternion_3", "quaternion_4", 
                        "calibstat_sys", "calibstat_gyro", "calibstat_accel", "calibstat_mag"]
    
        self.BNO_UPDATE_FREQUENCY_HZ = BNO_UPDATE_FREQUENCY_HZ
        self.exepath = path + '/measurement_system'
        self.datapath = path + '/data'
    
    
        self.INTERVAL = INTERVAL
        self.INIT_LEN = self.INTERVAL // self.BNO_UPDATE_FREQUENCY_HZ

        self.Time_data = deque(np.zeros(self.INIT_LEN))# Time
        self.linear_accel_x_data = deque(np.zeros(self.INIT_LEN))# linear_accel_x
        self.linear_accel_y_data = deque(np.zeros(self.INIT_LEN))# linear_accel_y
        self.linear_accel_z_data = deque(np.zeros(self.INIT_LEN))# linear_accel_z
        self.gyro_x_data = deque(np.zeros(self.INIT_LEN))# gyro_x
        self.gyro_y_data = deque(np.zeros(self.INIT_LEN))# gyro_y
        self.gyro_z_data = deque(np.zeros(self.INIT_LEN))# gyro_z
        
        self.assy_data = None
        
        self.i2c, self.bno = self.connect()

            
    def connect(self):
        i2c = busio.I2C(board.SCL, board.SDA)# I2CBusにアクセスするためのインターフェースを用意。SCLとSDAを使ってアクセス
        bno = adafruit_bno055.BNO055_I2C(i2c)# BNO055センサに接続する。DefaultではNDOF_MODE(12)
        #self.bno.use_external_crystal = True
        print("Established connection with BNO055")
        return i2c, bno
    
    def get_data(self, bno):
        
        '''
        センサからある時点の値を抽出
        '''
            
        euler_x, euler_y, euler_z = [val for val in bno.euler]# 角度[deg]
        gyro_x, gyro_y, gyro_z = [val for val in bno.gyro]# 角速度[rad/s]
        gravity_x, gravity_y, gravity_z = [val for val in bno.gravity]# 重力加速度[m/s^2]
        linear_accel_x, linear_accel_y, linear_accel_z = [val for val in bno.linear_acceleration]# 重力分を差し引いた加速度[m/s^2]
        accel_x, accel_y, accel_z = [val for val in bno.acceleration]# 重力+リニア加速度
        quaternion_1, quaternion_2, quaternion_3, quaternion_4 = [val for val in bno.quaternion]# クオータニオン
        calibstat_sys, calibstat_gyro, calibstat_accel, calibstat_mag = [val for val in bno.calibration_status]# キャリブレーション状態
            
        return euler_x, euler_y, euler_z, gyro_x, gyro_y, gyro_z, gravity_x, gravity_y, gravity_z,\
                linear_accel_x, linear_accel_y, linear_accel_z, accel_x, accel_y, accel_z,\
                quaternion_1, quaternion_2, quaternion_3, quaternion_4,\
                calibstat_sys, calibstat_gyro, calibstat_accel, calibstat_mag
        
    

In [3]:
def wait_process(wait_sec):
    until = perf_counter() + wait_sec
    while perf_counter() < until:
        pass
    return

In [4]:


class measurement_control:
    def __init__(self, BNO_UPDATE_FREQUENCY_HZ = 10, Isautosave = True):
        self.ctrlbutton_state = 'stop'# stop/start
        self.ctrl_button = widgets.Button(description="▶",layout=lay(120,50)) 
        self.save_button = widgets.Button(description="Save",layout=lay(120,50))
        self.output = widgets.Output(layour={'border': '1px solid black'})       
        self.measurement = measuremet(BNO_UPDATE_FREQUENCY_HZ=BNO_UPDATE_FREQUENCY_HZ)
        self.Isstart = False
        self.start_thread = None
        self.stop_thread = None
        self.save_thread = None
        self.buttonState = "stop"
        self.elapsed_time = 0
        self.itr_end_time = 0
        self.logic_end_time = 0
        self.logic_start_time = 0
        self.itr_start_time = 0
        self.Isautosave = Isautosave
        
    def start(self):
        
        self.Isstart = True
        self.stop_thread = None
        #print('Measurement Start')
        self.ctrl_button.description = '■'
        self.ctrl_button.button_style = 'success'
        
        
        #データ計測ロジック
        try:
                  
            #plt.ion()
            #fig, ax = plt.subplots(3, 1, tight_layout=True)
            #fig.set_figheight(8)
            #fig.set_figwidth(12)

            self.logic_start_time = time.time()#計測用
            counter = 0
            #self.logic_start_time = time.time()#計測用
            
            while self.Isstart:
                self.itr_start_time = time.time()# ループの一番最初
                Time = counter / self.measurement.BNO_UPDATE_FREQUENCY_HZ# 現在の時刻
                
                #Time, euler_x, euler_y, euler_z, gyro_x, gyro_y, gyro_z, gravity_x, gravity_y, gravity_z,\
                #linear_accel_x, linear_accel_y, linear_accel_z, accel_x, accel_y, accel_z,\
                #quaternion_1, quaternion_2, quaternion_3, quaternion_4,\
                #calibstat_sys, calibstat_gyro, calibstat_accel, calibstat_mag]
                                 
                #from IPython.core.debugger import Pdb; Pdb().set_trace()
                data = np.array([Time] + list(self.measurement.get_data(self.measurement.bno))).reshape(-1, len(self.measurement.COLUMNS))#センサからデータを抽出しTimeの結合
                
                            
                if counter == 0:
                    
                    self.measurement.assy_data = data.copy()

                else: 
                    self.measurement.assy_data = np.concatenate((self.measurement.assy_data, data), axis = 0)# 行方向に結合

                '''#グラフ化機構
                if self.measurement.BNO_UPDATE_FREQUENCY_HZ <= 1.0:
                
                    #Queue operation               
                    self.measurement.Time_data.popleft()
                    self.measurement.linear_accel_x_data.popleft()
                    self.measurement.linear_accel_y_data.popleft()
                    self.measurement.linear_accel_z_data.popleft()
                    self.measurement.gyro_x_data.popleft()
                    self.measurement.gyro_y_data.popleft()
                    self.measurement.gyro_z_data.popleft()
                    
                    self.measurement.Time_data.append(data[-1,0])
                    self.measurement.linear_accel_x_data.append(data[-1,10])
                    self.measurement.linear_accel_y_data.append(data[-1,11])
                    self.measurement.linear_accel_z_data.append(data[-1,12])
                    self.measurement.gyro_x_data.append(data[-1,4])
                    self.measurement.gyro_y_data.append(data[-1,5])
                    self.measurement.gyro_z_data.append(data[-1,6])         
                    
                    
                    # Graph
                    # 極力グラフを書くだけにとどめる
                    #clear_output(wait = True)
                    #plt.cla()
                    #ax[0].plot(self.measurement.Time_data,self.measurement.linear_accel_x_data,linestyle='-', color='r', linewidth=1)
                    #ax[1].plot(self.measurement.Time_data,self.measurement.linear_accel_y_data,linestyle='-', color='r', linewidth=1)
                    #ax[2].plot(self.measurement.Time_data,np.rad2deg(self.measurement.gyro_z_data),linestyle='-', color='r', linewidth=1)# グラフ化の時はrad2degにしておく
                    #display(fig)
                '''
                
                self.itr_end_time = time.time()# ある関数が終わった直後
   
                wait_process((1.0 / self.measurement.BNO_UPDATE_FREQUENCY_HZ) - (self.itr_end_time - self.itr_start_time))
                counter += 1

            self.logic_end_time = time.time()
            # 経過時間を出力(秒)
            self.elapsed_time = self.logic_end_time - self.logic_start_time         
            #print('elapsed_time:{0} s'.format(self.elapsed_time))#実行時間出力
                            
        except Exception as e:
            self.logic_end_time = time.time()
            # 経過時間を出力(秒)
            self.elapsed_time = self.logic_end_time - self.logic_start_time
            #print(self.elapsed_time)
            print('Error! Save data')
            self.save()
            return

        
    def stop(self):
        self.Isstart = False
        self.start_thread = None
        #print('Stop Measurement')
        try:
            if meas_ctrl.Isautosave:
                self.save()
            self.ctrl_button.description = '▶'
            self.ctrl_button.button_style = ''
        except Exception as e:
            print('Error!')
        
        return

    def save(self):
        self.save_button.description = 'Saving...'
        self.save_button.button_style = 'success'
        self.measurement.assy_data = pd.DataFrame(self.measurement.assy_data, columns=list(self.measurement.COLUMNS))
        
        t_delta = datetime.timedelta(hours=9)
        JST = datetime.timezone(t_delta, 'JST')
        now = datetime.datetime.now(JST)
        timestamp = now.strftime('%Y%m%d%H%M%S')
        self.measurement.assy_data.to_csv(self.measurement.datapath + '/'+ timestamp +'_data.csv')
        
        wait_process(0.5)# for user friendly
        
        self.save_button.description = 'Save'
        self.save_button.button_style = ''
        self.save_thread = None


    def Run(self):
        @self.output.capture()
        def on_click_ctrlbutton_callback(clicked_button: widgets.Button) -> None:
            if self.ctrlbutton_state == 'stop':
                if self.start_thread == None :
                    self.ctrlbutton_state = 'start'
                    self.start_thread = threading.Thread(target=self.start)
                    self.start_thread.start()
                
            elif self.ctrlbutton_state == 'start':
                if self.stop_thread == None :
                    self.ctrlbutton_state = 'stop'
                    self.stop_thread = threading.Thread(target=self.stop)
                    self.stop_thread.start()
                    
        def on_click_savebutton_callback(clicked_button: widgets.Button) -> None:
            if self.ctrlbutton_state == 'stop' and self.Isstart == False:
                self.save_thread = threading.Thread(target=self.save)
                self.save_thread.start()

        self.ctrl_button.on_click(on_click_ctrlbutton_callback)
        self.save_button.on_click(on_click_savebutton_callback)

        
def lay(width,height):
    return widgets.Layout(width=str(width)+"px",height=str(height)+"px")
def init_meas_system():
    meas_ctrl = measurement_control(BNO_UPDATE_FREQUENCY_HZ=10, Isautosave=False)
    display(widgets.VBox([
            widgets.HBox([meas_ctrl.ctrl_button, meas_ctrl.save_button]), 
            meas_ctrl.output]))
    return meas_ctrl
def main_loop():
    meas_ctrl = init_meas_system()
    meas_ctrl.Run()
    return meas_ctrl


In [9]:
meas_ctrl = main_loop()

Established connection with BNO055


In [10]:
#meas_ctrl.elapsed_time
# 計測ロジック全体の実行時間(データ出力を含まない)　データは0[s]からはじまるが、timeは0から始まらないので1サンプル分減算
meas_ctrl.logic_end_time - meas_ctrl.logic_start_time - (1.0 / meas_ctrl.measurement.BNO_UPDATE_FREQUENCY_HZ)


4.000413322448731

In [220]:
meas_ctrl.itr_end_time - meas_ctrl.itr_start_time# センサからデータを1回得るための時間。この時間がサンプリングレートより小さくないといけない 0.04sほど


0

In [221]:
(1.0 / meas_ctrl.measurement.BNO_UPDATE_FREQUENCY_HZ) - (meas_ctrl.itr_end_time - meas_ctrl.itr_start_time)# 次のイタレーションを実行するまでの待ち時間

0.1

In [222]:
# センサーからデータを一回得るための時間＋次のイタレーションまでの待ち時間 = サンプリングレート[s]
(meas_ctrl.itr_end_time - meas_ctrl.itr_start_time) + (1.0 / meas_ctrl.measurement.BNO_UPDATE_FREQUENCY_HZ) - (meas_ctrl.itr_end_time - meas_ctrl.itr_start_time)

0.1

In [223]:
(1.0 / meas_ctrl.measurement.BNO_UPDATE_FREQUENCY_HZ) # サンプリングレート[s]

0.1

In [224]:
# time関数の実行時間は0.00107sほど
start = time.time()
time.time()
end = time.time()
end - start


0.00023245811462402344

In [225]:
# print関数の実行時間は0.00138sほど
start = time.time()
print("計測停止ボタンが押されました")
end = time.time()
end - start


計測停止ボタンが押されました


0.0009047985076904297

In [226]:
# クラスメンバへのアクセスの実行時間は0.000206sほど
start = time.time()
meas_ctrl.measurement.BNO_UPDATE_FREQUENCY_HZ
end = time.time()
end - start

0.0002639293670654297